In [1]:
""":file: MEV_telephone_cross_fix.py

Version for Biogeme 3.2.13

:author: Michel Bierlaire
:date: Fri Apr 28 10:07:58 2023

"""

':file: MEV_telephone_cross_fix.py\n\nVersion for Biogeme 3.2.13\n\n:author: Michel Bierlaire\n:date: Fri Apr 28 10:07:58 2023\n\n'

In [2]:
import biogeme.biogeme as bio
from biogeme.expressions import Beta
from biogeme.models import logcnl
from biogeme.nests import OneNestForCrossNestedLogit, NestsForCrossNestedLogit

In [3]:
from telephone_data import (
    database,
    choice,
    avail1,
    avail2,
    avail3,
    avail4,
    avail5,
    logcostBM,
    logcostSM,
    logcostLF,
    logcostEF,
    logcostMF,
)

Parameters to be estimated
Arguments:
  1  Name for report. Typically, the same as the variable
  2  Starting value
  3  Lower bound
  4  Upper bound
  5  0: estimate the parameter, 1: keep it fixed

In [4]:
ASC_BM = Beta('ASC_BM', 0, None, None, 0)
ASC_EF = Beta('ASC_EF', 0, None, None, 0)
ASC_LF = Beta('ASC_LF', 0, None, None, 0)
ASC_MF = Beta('ASC_MF', 0, None, None, 0)
B_COST = Beta('B_COST', 0, None, None, 0)

Nest parameters

In [5]:
N_FLAT = Beta('N_FLAT', 1, None, None, 0)
N_MEAS = Beta('N_MEAS', 1, None, None, 0)

In [6]:
A_FLAT_LF = 0.5
A_MEAS_LF = 0.5

Define here arithmetic expressions for names that are not directly
available from the data

Utilities

In [7]:
V_BM = ASC_BM + B_COST * logcostBM
V_SM = B_COST * logcostSM
V_LF = ASC_LF + B_COST * logcostLF
V_EF = ASC_EF + B_COST * logcostEF
V_MF = ASC_MF + B_COST * logcostMF

In [8]:
V = {1: V_BM, 2: V_SM, 3: V_LF, 4: V_EF, 5: V_MF}
avail = {1: avail1, 2: avail2, 3: avail3, 4: avail4, 5: avail5}

Definitions of nests

In [9]:
alpha_N_FLAT = {1: 0, 2: 0, 3: A_FLAT_LF, 4: 1, 5: 1}
alpha_N_MEAS = {1: 1, 2: 1, 3: A_MEAS_LF, 4: 0, 5: 0}

In [10]:
nest_N_FLAT = OneNestForCrossNestedLogit(
    nest_param=N_FLAT, dict_of_alpha=alpha_N_FLAT, name='flat'
)
nest_N_MEAS = OneNestForCrossNestedLogit(
    nest_param=N_MEAS, dict_of_alpha=alpha_N_MEAS, name='measured'
)

In [11]:
nests = NestsForCrossNestedLogit(
    choice_set=list(V), tuple_of_nests=(nest_N_FLAT, nest_N_MEAS)
)

CNL with fixed alphas

In [12]:
logprob = logcnl(V, avail, nests, choice)
the_biogeme = bio.BIOGEME(database, logprob)
the_biogeme.modelName = 'MEV_telephone_cross_fix'
the_biogeme.calculateNullLoglikelihood(avail)
results = the_biogeme.estimate()

Get the results in a pandas table

In [13]:
pandasResults = results.getEstimatedParameters()
print(pandasResults)

           Value  Rob. Std err  Rob. t-test  Rob. p-value
ASC_BM -1.635755      0.530693    -3.082298  2.054088e-03
ASC_EF  2.426337      1.630259     1.488314  1.366681e-01
ASC_LF  2.199657      0.638822     3.443300  5.746611e-04
ASC_MF  4.084708      1.166941     3.500354  4.646411e-04
B_COST -3.354166      0.548565    -6.114433  9.690115e-10
N_FLAT  0.510742      0.205475     2.485670  1.293079e-02
N_MEAS  0.393418      0.102387     3.842454  1.218100e-04


In [14]:
pandasResults

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_BM,-1.635755,0.530693,-3.082298,2.054088e-03
ASC_EF,2.426337,1.630259,1.488314,1.366681e-01
ASC_LF,2.199657,0.638822,3.443300,5.746611e-04
ASC_MF,4.084708,1.166941,3.500354,4.646411e-04
B_COST,-3.354166,0.548565,-6.114433,9.690115e-10
N_FLAT,0.510742,0.205475,2.485670,1.293079e-02
N_MEAS,0.393418,0.102387,3.842454,1.218100e-04


In [15]:
print(f'Nbr of observations: {database.getNumberOfObservations()}')
print(f'LL(0) =    {results.data.initLogLike:.3f}')
print(f'LL(beta) = {results.data.logLike:.3f}')
print(f'AIC = {results.data.akaike:.1f}')
print(f'Output file: {results.data.htmlFileName}')

Nbr of observations: 434
LL(0) =    -473.606
LL(beta) = -473.606
AIC = 961.2
Output file: MEV_telephone_cross_fix~00.html
